In [1]:
import sys

!which {sys.executable}

/ut3/jerome/miniconda3/envs/wakai/bin/python


In [1]:
import sys
sys.path.insert(0, '../code')

# CTOI catalog

In [3]:
from catalog import get_ctois

ctois = get_ctois(clobber=True)
# tois.columns.tolist()

In [4]:
ctois.tail()

,TIC ID,CTOI,Promoted to TOI,MASTER,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Eff Temp (K) err,Stellar log(g) (cm/s^2),Stellar log(g) (cm/s^2) err,Stellar Radius (R_Sun),Stellar Radius (R_Sun) err,CTOI lastmod,User,Group,Tag,Notes
3660,1961935435,1.961935e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-09-07 08:21:56,nardiello,NaN,99732,"new CTOI likely member of NGC 7024, PATHOS-81 ..."
3661,2015243161,2.015243e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,128.000,4.07962,NaN,1.86272,NaN,2021-09-07 08:21:56,nardiello,NaN,99733,"new CTOI likely member of NGC 7245, PATHOS-82 ..."
3662,2025895835,2.025896e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023-10-15 12:57:33,conzo,NaN,432964,Transit detected from TESS-HUNTER Project
3663,2040889915,2.040890e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,132.842,4.39699,0.078333,1.09460,0.053869,2022-10-20 09:03:27,montalto,NaN,421032,"new CTOI, Montalto (2022)"
3664,2050290974,2.050291e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,369.772,4.44798,0.105034,1.01643,0.055942,2022-10-20 09:04:15,montalto,NaN,421311,"new CTOI, Montalto (2022)"


In [6]:
ctois[ctois.Notes.apply(lambda x: 'young' in str(x))]

,TIC ID,CTOI,Promoted to TOI,MASTER,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Eff Temp (K) err,Stellar log(g) (cm/s^2),Stellar log(g) (cm/s^2) err,Stellar Radius (R_Sun),Stellar Radius (R_Sun) err,CTOI lastmod,User,Group,Tag,Notes
204,25314899,2.531490e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.919824,NaN,2020-01-02 17:49:14,damboiu,NaN,8510,Host star is likely to be very young
708,88785435,8.878544e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,131.537,4.29328,0.126790,0.933972,0.102854,2024-01-19 03:43:56,vach,NaN,437419,Identified as part of young planet search (Vac...
1188,150070085,1.500701e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,144.714,4.44666,0.084384,1.054410,0.057211,2024-01-19 04:09:01,vach,NaN,437428,Identified as part of young planet search (Vac...
3403,434398831,4.343988e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,142.886,4.50503,0.080712,0.915558,0.050472,2024-01-19 03:50:41,vach,NaN,437422,Identified as part of young planet search (Vac...
3404,434398831,4.343988e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,142.886,4.50503,0.080712,0.915558,0.050472,2024-01-19 03:56:18,vach,NaN,437425,Identified as part of young planet search (Vac...


In [7]:
ctois.shape

(3655, 71)

# cross-match CTOI with Gaia DR3 using `astroquery.xmatch`

In [8]:
import astropy.units as u
from astroquery.xmatch import XMatch
import pandas as pd

gaia_table = XMatch.query(
    cat1=open('../data/CTOIs.csv'),
    cat2="vizier:I/355/gaiadr3",  # The Vizier identifier for Gaia DR3
    max_distance=1 * u.arcsec,    # Set the maximum matching radius
    colRA1="RA",  # Column name for Right Ascension in TIC results
    colDec1="Dec" # Column name for Declination in TIC results
)
gaia_df = gaia_table.to_pandas()

# results to 6237 matches
matches = gaia_df.copy()

# Apply magnitude filtering criteria
# For main-sequence stars, the difference between TESS T and Gaia G is around 0–0.5 magnitudes
# Redder stars might have T - G values closer to 1–2 magnitudes.
# For giants or highly reddened stars, the difference could be larger.
if False:
    # results to 5971 matches
    matches = []
    for i, row in gaia_df.iterrows():
        teff=row['Stellar Eff Temp (K)']
        if teff<3500:
            magnitude_threshold = 2
        elif (teff>3500) and (teff<6000):
            magnitude_threshold = 1
        else:
            magnitude_threshold = 1.5
        if abs(row['Gmag'] - row['TESS Mag']) < magnitude_threshold:
            matches.append(row)
    matches = pd.concat(matches, axis=1).T
if False:
    # results to 5700 matches
    magnitude_threshold = 1
    matches = gaia_df[abs(gaia_df['Gmag'] - gaia_df['TESS Mag']) < magnitude_threshold]
if False:
    # results to 4878 matches
    teff_threshold = 1000
    matches = gaia_df[abs(gaia_df['Teff'] - gaia_df['Stellar Eff Temp (K)']) < teff_threshold]
matches.tail()

Could not import regions, which is required for some of the functionalities of this module.


,angDist,TIC ID,CTOI,Promoted to TOI,MASTER,SG1A,SG1B,SG2,SG3,SG4,...,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
3683,0.001544,1961935435,1.961935e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,316.461056,41.547444,0.056352,0.200240,0.8499
3684,0.004345,2015243161,2.015243e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.3057,0.6911,0.6834,0.7013,A,333.824303,54.296077,0.417660,0.385990,-0.0473
3685,0.010442,2025895835,2.025896e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,327.057880,-78.471855,1.459336,1.967866,-0.1915
3686,0.005400,2040889915,2.040890e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0098,0.0022,0.0005,0.0053,MARCS,358.841348,41.359560,0.201653,0.180558,0.0696
3687,0.016903,2050290974,2.050291e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.5539,0.2987,0.2979,0.2996,MARCS,346.183963,68.515237,0.175257,0.167157,-0.0455


In [9]:
matches.shape

(3688, 209)

In [11]:
matches['GaiaDR3'] = matches['DR3Name'].apply(lambda x: int(x.split()[-1]))
matches.to_csv('../data/CTOIs_GaiaDR3_xmatch.csv', index=False)

In [19]:
import pandas as pd

#https://zenodo.org/records/8250524
df = pd.read_csv('../data/Perren2023/UCC_cat.csv.gz')
df.tail()

,ID,RA_ICRS,DE_ICRS,plx,pmRA,pmDE,UCC_ID,N_50,r_50,RA_ICRS_m,DE_ICRS_m,plx_m,pmRA_m,pmDE_m,Rv_m,N_Rv,C1,C2,C3
16174,MKN 1;MKN ab,266.4468,-29.0874,NaN,NaN,NaN,UCC G359.8-00.1,79.0,4.4,266.462,-29.095,0.489,1.753,-0.777,-25.868,5.0,0.86,0.75,AA
16175,MWSC 5326,248.1925,-17.1430,NaN,NaN,NaN,UCC G359.9+20.4,0.0,4.6,248.189,-17.148,0.090,-3.735,-5.860,67.828,1.0,0.11,0.20,DD
16176,HSC 2986,285.3171,-36.8834,6.694,2.209,-27.556,UCC G359.9-17.7,0.0,22.5,285.298,-36.941,6.532,3.670,-27.737,-5.275,8.0,0.72,1.00,BA
16177,VVV 149;VVV-CL 149,267.3380,-29.4640,NaN,NaN,NaN,UCC G359.9-00.9,0.0,2.6,267.351,-29.455,0.352,-0.674,-2.780,-39.904,3.0,0.45,0.46,CC
16178,Ryu 535,266.5884,-29.0573,NaN,NaN,NaN,UCC G359.9-00.2,61.0,2.1,266.606,-29.055,0.376,-0.153,-2.099,-33.921,4.0,0.88,0.31,AC


In [20]:
df_mem = pd.read_parquet('../data/Perren2023/UCC_members.parquet.gz')
df_mem.tail()

,name,Source,RA_ICRS,DE_ICRS,Plx,e_Plx,pmRA,e_pmRA,pmDE,e_pmDE,RV,e_RV,GLON,GLAT,Gmag,BP-RP,e_Gmag,e_BP-RP,probs
1337412,ryu30,5933719906152539904,247.228085,-52.699637,0.240993,0.158018,-4.525831,0.154389,-4.983059,0.127764,NaN,NaN,332.477863,-2.781450,18.330563,2.435881,0.003246,0.088112,0.46891
1337413,ryu30,5933720318469495424,247.180385,-52.706245,0.135389,0.053207,-4.841031,0.055405,-5.292475,0.045611,NaN,NaN,332.453112,-2.765083,16.444132,1.731100,0.002817,0.009220,0.45149
1337414,ryu30,5933720107969318016,247.178643,-52.736859,0.300294,0.121510,-4.620295,0.120153,-5.140017,0.103496,NaN,NaN,332.430199,-2.785444,17.966627,1.370831,0.002976,0.017981,0.45149
1337415,ryu30,5933720490268392704,247.119139,-52.692309,0.270647,0.230620,-4.524732,0.220578,-4.939661,0.182071,NaN,NaN,332.437554,-2.728610,18.863051,2.578410,0.003619,0.063381,0.41418
1337416,ryu30,5933730587694794880,247.078632,-52.721215,0.231349,0.169833,-4.648944,0.173652,-4.778104,0.143794,NaN,NaN,332.399658,-2.730834,18.484586,NaN,0.003323,NaN,0.40547


In [36]:
matches['Source'] = matches.GaiaDR3.copy()

In [39]:
idx1 = df_mem.Source.isin(matches.Source)
idx.sum()

85

In [40]:
idx2 = matches.GaiaDR3.isin(df_mem.Source)
idx.sum()

85

In [41]:
matches[idx2]['Promoted to TOI'].unique()

array([    nan, 1881.01, 6650.01, 6650.02, 6650.03, 6650.04, 1772.02,
       6047.01, 6248.01, 1918.01, 1882.01, 6715.01, 2959.01])

In [44]:
final = pd.merge(df_mem[idx1], matches[idx2], on='Source')
final.tail()

,name,Source,RA_ICRS,DE_ICRS,Plx_x,e_Plx_x,pmRA_x,e_pmRA_x,pmDE_x,e_pmDE_x,...,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000,GaiaDR3
97,melotte102,5251740294225360128,155.375052,-63.736122,6.599237,0.031833,-16.286477,0.038538,11.471132,0.033183,...,0.4209,0.4153,0.4264,MARCS,155.375216,-63.736173,0.617823,0.532510,0.0802,5251740294225360128
98,ngc3114,5256717749007641344,150.368270,-59.851710,0.992110,0.011746,-7.318495,0.015827,4.017288,0.013312,...,NaN,NaN,NaN,NaN,150.368334,-59.851728,0.252257,0.211518,0.2635,5256717749007641344
99,ngc3114,5256502966294604800,151.247606,-60.302639,0.928298,0.027592,-7.524458,0.034065,3.556803,0.029688,...,0.2554,0.2470,0.2647,MARCS,151.247673,-60.302655,0.546114,0.478389,0.2495,5256502966294604800
100,oc0609,5878452576220299520,217.691192,-60.776521,0.431238,0.015724,-5.837465,0.014847,-3.135288,0.017640,...,NaN,NaN,NaN,NaN,217.691245,-60.776507,0.240552,0.283707,0.0187,5878452576220299520
101,bica449,5333821173682843648,172.541443,-63.132304,0.391810,0.015461,-6.444174,0.016855,1.042715,0.015725,...,0.4748,0.4723,0.4765,A,172.541506,-63.132308,0.272845,0.252295,0.1671,5333821173682843648


In [45]:
final.to_csv('../../toi-viz/data/CTOI/ctoi_in_Perren2023.csv', index=False)

In [34]:
matches[idx2]['TIC ID'].to_csv('../../toi-viz/data/CTOI/ctoi_in_Perren2023_ticids.txt', 
                               index=False, header=False)